In [ ]:
import json
import os
import openai
import jsonlines
from tqdm import tqdm
import time
import argparse
import pandas as pd
import numpy as np
import torch
# model_name = "gpt-4o" # "gpt-3.5-turbo-0125"
# openai.api_key = ''
from langchain_text_splitters import RecursiveCharacterTextSplitter

1. gpt_template <- get_template(paper)
2. QAs <- gpt4o(gpt_template)
3. database mapping: map chunk of papers into chunk_id
4. gold passage mapping: map rationale of QAs into chunk_id 
5. build bm25 & embedding space



In [ ]:
# wiki = pd.read_csv("/data/../nlp_data/topiocqa/full_wiki_segments.tsv",
#                    sep='\t', nrows=2000)
wiki = pd.read_csv("/data/../nlp_data/topiocqa/full_wiki_segments.tsv",
                   sep='\t')

In [ ]:
# average length of each chunk
wiki.head()

In [ ]:
sampled_wiki=wiki.sample(300)
inds = list(sampled_wiki.index)
lengths = [len(sampled_wiki.loc[ind, 'text']) for ind in inds]
sum(lengths)/len(lengths)


In [ ]:
lengths[:10]

In [ ]:
# english paper 

In [ ]:
en_papers = torch.load("../../nlp_data/kisti/db_files/en_paperpath2lang_cnt")
paper2lang_cnt = torch.load("../../nlp_data/kisti/db_files/paperpath2lang_cnt")
paperpath2conv = torch.load("../../nlp_data/kisti/db_files/paperpath2gen_conv")


long_enough_ps = [f_path for f_path in list(en_papers.keys()) if len(extract_content_path(f_path))>15000]
len(long_enough_ps)

gen_papers = list(paperpath2conv.keys())
# gen_papers

In [ ]:
len(en_papers), len(long_enough_ps)

In [ ]:
def extract_content_path(path):
    f = open(path) # certain_domain_en_ps[5]
    # print(path)
    body = json.load(f,)['body_text']
    all_content = ""
    # print(body)
    prev_section_name = ""
    for section in body:

        if 'text' in section:
            section_name = section['section'] if 'section' in section else ""
            for text in section['text']:
                if prev_section_name:
                    # print(prev_section_name+": "+section_name+": "+ text)
                    all_content += prev_section_name+": "+section_name+": "+ text + "\n"
                else:
                    # print(section_name+": "+ text)
                    all_content += section_name+": "+ text + "\n"
            prev_section_name = ""
        else:
            prev_section_name = section['section']
    # print()
    # len(all_content)
    return all_content

# long_enough_ps = [f_path for f_path in certain_domain_en_ps if len(extract_content_path(f_path))>15000]
# len(long_enough_ps)

# import random
# random.seed(0)
# sampled_papers = random.sample(long_enough_ps, 900)

In [ ]:
import re

def extract_rationale(text):
    """Extracts the rationale part from the given text."""
    rationale_pattern = re.compile(r'(?:rationale:|Rationale:|\*rationale\*|\*Rationale\*|_Rationale_:)(.*?)(?=\n|$)', 
                                   re.IGNORECASE | re.DOTALL)
    rationales = rationale_pattern.findall(text)
    return [clean_rationale(rationale.strip()) for rationale in rationales]
def clean_rationale(rationale):
    """Cleans up the extracted rationale text."""
    # Remove leading formatting characters like "** " or any other unwanted characters
    return re.sub(r'^\*\*\s*', '', rationale)

def extract_q(text):
    """Extracts the rationale part from the given text."""
    rationale_pattern = re.compile(r'(?:rationale:|Rationale:|\*rationale\*|\*Rationale\*|_Rationale_:)(.*?)(?=\n|$)', 
                                   re.IGNORECASE | re.DOTALL)
    rationales = rationale_pattern.findall(text)
    return [clean_rationale(rationale.strip()) for rationale in rationales]
def clean_q(rationale):
    """Cleans up the extracted rationale text."""
    # Remove leading formatting characters like "** " or any other unwanted characters
    return re.sub(r'^\*\*\s*', '', rationale)
# rats = extract_rationale(paperpath2conv[gen_papers[3]]["choices"][0]["message"]["content"])
# len(rats), rats

### get rationales

In [ ]:
gen_paperpath2rationales = {}
for paper_path in gen_papers:
    rats = extract_rationale(paperpath2conv[paper_path]["choices"][0]["message"]["content"])
    gen_paperpath2rationales[paper_path] = rats
    # len(rats), rats

In [ ]:
len_rats = torch.tensor([len(gen_paperpath2rationales[paper_path]) for i,paper_path in enumerate(gen_papers)])
zero_rat_paths = np.array(gen_papers)[len_rats == 0]
print(len(zero_rat_paths))
# print(paperpath2conv[zero_rat_paths[0]]["choices"][0]["message"]["content"])
if zero_rat_paths:
    print(paperpath2conv[zero_rat_paths[0]]["choices"][0]["message"]["content"])


### get Q&A

In [ ]:
# print(paperpath2conv[gen_papers[0]]["choices"][0]["message"]["content"][:500])
# print()
# print(paperpath2conv[gen_papers[150]]["choices"][0]["message"]["content"][:500])
# print()
# print(paperpath2conv[gen_papers[180]]["choices"][0]["message"]["content"][:500])
# print()
# print(paperpath2conv[gen_papers[250]]["choices"][0]["message"]["content"][:500])

In [ ]:
The following text is a response from an AI chatbot. Please retain the exact meaning of the subsequent sentence but rephrase it as much as possible, ensuring that the key terms remain unchanged.



In [ ]:
print(paperpath2conv[gen_papers[1]]["choices"][0]["message"]["content"][:4000])


In [ ]:
import re

def extract_questions(text):
    """Extracts the questions (Q) from the given text."""
    # Regular expression pattern to match questions in various formats
    question_pattern = re.compile(
        r'(?:(?<![\w(])(?:\*\*)?Q\d+(?:\s*:|\s*:\s*|\s*|\*\*:\s*)(.*?)(?=\n|$)|'
        r'(?:\*\*Questioner\*\*:)(.*?)(?=\n|$)|'
        r'\*\*Questioner \(Q1\):\*\*\s*(.*?)(?=\n|$))', 
        re.IGNORECASE | re.DOTALL
    )
            # r'(?:\*\*Questioner\s*\(Q1\)\*\*:)(.*?)(?=\n|$))', 
    questions = question_pattern.findall(text)
    # Flatten the list of tuples and filter out empty strings
    questions = [item for sublist in questions for item in sublist if item]
    return [clean_question(q.strip()) for q in questions]

def clean_question(question):
    """Cleans up the extracted question text."""
    # Remove leading formatting characters like "** " or any other unwanted characters
    return re.sub(r'^\*\*\s*|\*\*$', '', question).strip()

def extract_answers(text):
    """Extracts the questions (Q) from the given text."""

    answer_pattern = re.compile(
        r'(?:(?:\n\s*)(?:\*\*)?A\d+(?:\s*:|\s*:\s*|\s*|\*\*:\s*)(.*?)(?=\n|$)|'
        r'(?:\*\*Answerer\*\*:\s*)(.*?)(?=\n|$)|'
        r'\*\*Answerer \(A\d+\):\*\*\s*(.*?)(?=\n|$))', 
        re.IGNORECASE | re.DOTALL
    )
       
    answers = answer_pattern.findall(text)
    # Flatten the list of tuples and filter out empty strings
    answers = [item for sublist in answers for item in sublist if item]
    return [clean_answer(a.strip()) for a in answers]

def clean_answer(answer):
    """Cleans up the extracted question text."""
    # Remove leading formatting characters like "** " or any other unwanted characters
    return re.sub(r'^\*\*\s*|\*\*$', '', answer).strip()

# answers = extract_answers(paperpath2conv[gen_papers[150]]["choices"][0]["message"]["content"])
# for a in answers:
#     print(a)


In [ ]:
gen_paperpath2Qs = {}
for paper_path in gen_papers:
    qs = extract_questions(paperpath2conv[paper_path]["choices"][0]["message"]["content"])
    gen_paperpath2Qs[paper_path] = qs
    # len(rats), rats
    
gen_paperpath2As = {}
for paper_path in gen_papers:
    ans = extract_answers(paperpath2conv[paper_path]["choices"][0]["message"]["content"])
    gen_paperpath2As[paper_path] = ans
    # len(rats), rats

len_qs = torch.tensor([len(gen_paperpath2Qs[paper_path]) for i,paper_path in enumerate(gen_papers)])
zero_q_paths = np.array(gen_papers)[len_qs == 0]
print(len(zero_q_paths))
# print(paperpath2conv[zero_rat_paths[0]]["choices"][0]["message"]["content"])
if zero_q_paths:
    print(paperpath2conv[zero_q_paths[0]]["choices"][0]["message"]["content"])

len_as = torch.tensor([len(gen_paperpath2As[paper_path]) for i,paper_path in enumerate(gen_papers)])
zero_a_paths = np.array(gen_papers)[len_as == 0]
print(len(zero_a_paths))
# print(paperpath2conv[zero_rat_paths[0]]["choices"][0]["message"]["content"])
if zero_a_paths:
    print(paperpath2conv[zero_a_paths[0]]["choices"][0]["message"]["content"])


In [ ]:
len(gen_paperpath2rationales), sum([len(rs) for rs in list(gen_paperpath2rationales.values())])

In [ ]:
len(gen_paperpath2Qs),sum([len(qs) for qs in list(gen_paperpath2Qs.values())])

In [ ]:
len(gen_paperpath2As),sum([len(ans) for ans in list(gen_paperpath2As.values())])

In [ ]:
torch.save(paperpath2conv, "../../nlp_data/kisti/db_files/paperpath2gen_conv_v2")


### DB data chunking

In [ ]:

def get_chunks_from_path(paper_path, max_n_letters=10000):
    # print(extract_content_path(ex_paper_path))
    content=extract_content_path(paper_path)[:max_n_letters]

    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n", ". "],  # Secondary and tertiary split markers
        keep_separator=False,
        chunk_size=500,  # Adjust the chunk size as needed
        chunk_overlap=100  # Adjust the overlap as needed
    )

    # Split the preprocessed content
    chunks = text_splitter.split_text(content)
    # print([len(chunk) for chunk in chunks])
    return chunks

chunks = get_chunks_from_path(long_enough_ps[0])
len(chunks), chunks,

In [ ]:
paperpath2chunks = {}
for paper_path in long_enough_ps:
    chunks = get_chunks_from_path(paper_path)
    paperpath2chunks[paper_path] = chunks

In [ ]:
len(paperpath2chunks), len(long_enough_ps)

In [ ]:
# Initialize an empty list to store rows
rows = []

# Iterate over the dictionary
for path, chunks in paperpath2chunks.items():
    for i,chunk in enumerate(chunks):
        # Append a tuple (path, chunk) to the list of rows
        rows.append((path, i, path+'_'+str(i), chunk))

# Create a DataFrame from the list of rows
df = pd.DataFrame(rows, columns=['path', 'chunk_id', 'path_chunk_id', 'text'])
df['id'] = df.index
df = df[['id', 'path_chunk_id', 'path','text']]
# Display the DataFrame
df.head()


In [ ]:
df.shape, df['path_chunk_id'][13:20].values, 

In [ ]:
torch.save(df, "../../nlp_data/kisti/db_files/papers_chunks")


In [ ]:
# import pandas as pd
import json
import os
from tqdm import tqdm

# Step 2: Define the number of rows per segment
rows_per_segment = 50000  # Adjust this number based on your needs

# Step 3: Split the DataFrame into smaller segments and write each to a .jsonl file
# "/data/../nlp_data/preprocessed"
# "/data/../nlp_data/collection-paragraph"
output_dir = "../../nlp_data/kisti/collection-paragraph-kisti"
os.makedirs(output_dir, exist_ok=True)

num_segments = (len(df) + rows_per_segment - 1) // rows_per_segment  # Calculate the number of segments
print("num_segments: ", num_segments)

for i in tqdm(range(num_segments)):
    start_row = i * rows_per_segment
    end_row = min((i + 1) * rows_per_segment, len(df))
    segment = df.iloc[start_row:end_row]

    jsonl_file_path = os.path.join(output_dir, f'segment_{i + 1}.jsonl')
    with open(jsonl_file_path, 'w') as jsonl_file:
        for index, row in segment.iterrows():
            entry = {
                "id": row['path_chunk_id'],
                "contents": row['text']
                # "contents": row['title'].replace(" [SEP]", ":") + ": " + row['text']
                # row['text']
            }
            jsonl_file.write(json.dumps(entry) + '\n')

    # print(f'Segment {i + 1} written to {jsonl_file_path}')

In [ ]:
39*50000

In [ ]:
len(df),len(df)/8

In [ ]:
def longest_common_substring(str1, str2):
    """Returns the length of the longest common substring between two strings."""
    m = len(str1)
    n = len(str2)
    # Create a table to store lengths of longest common suffixes of substrings.
    # Note that LCSuff[i][j] contains the length of the longest common suffix
    # of str1[0...i-1] and str2[0...j-1].
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)]
    result = 0  # To store length of the longest common substring.

    # Building the LCSuff table in a bottom-up fashion.
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                LCSuff[i][j] = 0
            elif str1[i-1] == str2[j-1]:
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j])
            else:
                LCSuff[i][j] = 0
    return result

def find_element_with_longest_overlap(list_of_strings, target_string):
    """Finds the element in the list with the longest overlap with the target string."""
    max_overlap = 0
    best_match = None

    for i,element in enumerate(list_of_strings):
        overlap_length = longest_common_substring(element, target_string)
        if overlap_length > max_overlap:
            max_overlap = overlap_length
            best_match = i,element

    return best_match


# target_string = "Prolactin (PRL) is a polypeptide hormone that is synthesized in the pituitary gland, consists of 199 amino acids with a molecular mass of 23KD, and has more functions than all other pituitary hormones combined... The initial step in the action of PRL, similar to all other hormones, is binding to the extracellular domain of prolactin receptor (PRLR)."

# best_match = find_element_with_longest_overlap(chunks, target_string)
# print("Element with the longest overlap:", best_match)


In [ ]:
paperpath2matches = {}
for paper_path, rationales in gen_paperpath2rationales.items():
    chunks = paperpath2chunks[paper_path]
    matches = []
    for target_string in rationales:
        best_match = find_element_with_longest_overlap(chunks, target_string)
        # print("Element with the longest overlap:", best_match)
        matches += [best_match[0]]
    paperpath2matches[paper_path] = matches

In [ ]:
torch.save(paperpath2matches, "../../nlp_data/kisti/db_files/gen_conv2matches")


In [ ]:
len(paperpath2matches),sum([len(matches) for matches in list(paperpath2matches.values())])

In [ ]:
exp_paper = list(paperpath2matches.keys())[0]
# paperpath2matches[exp_paper]

for rat, chunk_id in zip(gen_paperpath2rationales[exp_paper], paperpath2matches[exp_paper]):
    print("rat: ", rat)
    print("chunk: ",paperpath2chunks[exp_paper][chunk_id])
    print("####"*10 )
    
    

### create conv data as json 

In [ ]:
# train_paperpath
total_paperpaths = set(list(gen_paperpath2Qs.keys()))
len(total_paperpaths)

In [ ]:
import random
random.seed(2)
val_paperpaths = set(random.sample(list(total_paperpaths), 150))
train_paperpath = total_paperpaths-val_paperpaths
len(train_paperpath), len(val_paperpaths)

sum([len(gen_paperpath2Qs[p]) for p in list(train_paperpath)]), sum([len(gen_paperpath2Qs[p]) for p in list(val_paperpaths)])

In [ ]:
train_dt = []
for conv_id, path in enumerate(train_paperpath, 1):
    Qs, As, Rs = gen_paperpath2Qs[path], gen_paperpath2As[path], gen_paperpath2rationales[path]
    history_qs, history_as = [], []
    for turn_id, (q,a,r) in enumerate(zip(Qs, As, Rs), 1):
        sample_id = str(conv_id)+"-"+str(turn_id)
        instance = {'id':sample_id, 'conv_id':conv_id, 'turn_id':turn_id, 'query':q, 'answer':a,
                     'history_query':deepcopy(history_qs), 'history_answer':deepcopy(history_as)}
        # ['id', 'conv_id', 'turn_id', 'query', 'answer', 'history_query', 'history_answer',
        # 'pos_docs', 'pos_docs_id']
        history_qs += [q]
        history_as += [a]
        train_dt += [instance]
    
val_dt = []
for conv_id, path in enumerate(val_paperpaths, 1):
    Qs, As, Rs = gen_paperpath2Qs[path], gen_paperpath2As[path], gen_paperpath2rationales[path]
    history_qs, history_as = [], []
    for turn_id, (q,a,r) in enumerate(zip(Qs, As, Rs), 1):
        sample_id = str(conv_id)+"-"+str(turn_id)
        instance = {'id':sample_id, 'conv_id':conv_id, 'turn_id':turn_id, 'query':q, 'answer':a,
                     'history_query':deepcopy(history_qs), 'history_answer':deepcopy(history_as)}
        # ['id', 'conv_id', 'turn_id', 'query', 'answer', 'history_query', 'history_answer',
        # 'pos_docs', 'pos_docs_id']
        history_qs += [q]
        history_as += [a]
        val_dt += [instance]
        
len(train_dt), len(val_dt)

In [ ]:
root = "../../nlp_data/kisti"
split = "train"
with open(os.path.join(root,f'{split}_new'+".json"), 'w', encoding='utf-8') as file:
    for item in train_dt:
        # Convert the dictionary to a JSON string and write it to the file
        json_str = json.dumps(item)
        file.write(json_str)
        # if item != data[-1]:  # Check if it's not the last item
        #     file.write(",\n")  # For the last item, we don't add a comma
        # else:
        file.write("\n")

root = "../../nlp_data/kisti"
split = "dev"
with open(os.path.join(root,f'{split}_new'+".json"), 'w', encoding='utf-8') as file:
    for item in val_dt:
        # Convert the dictionary to a JSON string and write it to the file
        json_str = json.dumps(item)
        file.write(json_str)
        # if item != data[-1]:  # Check if it's not the last item
        #     file.write(",\n")  # For the last item, we don't add a comma
        # else:
        file.write("\n")

# print(f"Data saved to '{file_path}' successfully.")

In [ ]:
len(gen_paperpath2As)

In [ ]:
gen_paperpath2As[list(gen_paperpath2As.keys())[0]]

### create qrels for conv data as trec

In [ ]:
qrels_train = []
for conv_id, path in enumerate(train_paperpath, 1):
    # Qs, As, Rs = gen_paperpath2Qs[path], gen_paperpath2As[path], gen_paperpath2rationales[path]
    matches = paperpath2matches[path]
    for i, match_id in enumerate(matches, 0):
        turn_id = i+1
        sample_id = str(conv_id)+"-"+str(turn_id)
        path_chunk_id = path+'_'+str(match_id)
        path_chunk_id = '/'.join(path_chunk_id.split("/")[6:])
        path_chunk_id = path_chunk_id.replace(" ", "")
        qrel = f"{sample_id} Q0 {path_chunk_id} 1"
        
        qrels_train += [qrel]


# Save to a .trec file
with open('../../nlp_data/kisti/train_gold.trec', 'w') as f:
    for line in qrels_train:
        f.write(line + '\n')


In [ ]:
qrels_train = []
for conv_id, path in enumerate(val_paperpaths, 1):
    # Qs, As, Rs = gen_paperpath2Qs[path], gen_paperpath2As[path], gen_paperpath2rationales[path]
    matches = paperpath2matches[path]
    for i, match_id in enumerate(matches, 0):
        turn_id = i+1
        sample_id = str(conv_id)+"-"+str(turn_id)
        path_chunk_id = path+'_'+str(match_id)
        path_chunk_id = '/'.join(path_chunk_id.split("/")[6:])
        path_chunk_id = path_chunk_id.replace(" ", "")
        qrel = f"{sample_id} Q0 {path_chunk_id} 1"
        
        qrels_train += [qrel]

# Save to a .trec file
with open('../../nlp_data/kisti/dev_gold.trec', 'w') as f:
    for line in qrels_train:
        f.write(line + '\n')
